# 1. Importing Libraries

In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 2.Reading the Data

In [4]:
PROJECT_DIR  = r"D:\Analytics\Projects\FlightSageMakerProject"
DATA_DIR = "data"

In [5]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    return pd.read_csv(file_path)

In [6]:
flights = get_data("flight_price")
flights

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [7]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset contains 10683 columns
- Columns "Route" and "Total_Stops" have a missing value each
- The data type of some features is not appropriate

# Preliminary Analysis

### 3.1 Data types of features

In [8]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [9]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [10]:
value = flights.Date_of_Journey.iloc[6]
value

'12/03/2019'

In [11]:
Dep = flights.Dep_Time.iloc[6]
Arrive = flights.Arrival_Time.iloc[6]
print(type(Dep), type(Arrive))

<class 'str'> <class 'str'>


In [12]:
Dur = flights.Duration.iloc[6]
Dur

'15h 30m'

In [13]:
Stops = flights.Total_Stops.iloc[6]
Stops

'1 stop'

In [14]:
# Should be int and is already int so no changes required
Price = flights.Price.iloc[6]
Price

np.int64(11087)

### 3.2 Check for duplicates

In [ ]:
flights.duplicated(keep = False).sum()

(
    flights
    .loc[flights.duplicated(keep = False)]
    .sort_values(["Airline", "Date_of_Journey",	"Source",	"Destination"])
)

### Observations

- The type of 'Date_of_Journey', 'Dep_Time' and	'Arrival_Time' should be date-time
- The type of 'Duration' and 'Total_Stops' is mixed and it should be only numeric (ie only the number of minutes)
- There are 220 duplicates which should be removed

## 4. Detailed Analysis

### Airline

In [17]:
flights.Airline

0             IndiGo
1          Air India
2        Jet Airways
3             IndiGo
4             IndiGo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [18]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- The feature has inconsistent values

In [19]:
(
    flights
    .Airline
    .str.replace(" Premium economy", "")
    .str.replace(" Business", "")
    .str.replace("GoAir", "Go Air")
    .str.title()
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Go Air', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

### Date of Journey

In [28]:
pd.to_datetime(flights.Date_of_Journey, format='%d/%m/%Y')

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

- data type needs to be converted to date time

### Source

In [44]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

- Bangalore spelling is wrong

In [46]:
(
    flights
    .Source
    .replace("Banglore", "Bangalore")
)

0        Bangalore
1          Kolkata
2            Delhi
3          Kolkata
4        Bangalore
           ...    
10678      Kolkata
10679      Kolkata
10680    Bangalore
10681    Bangalore
10682        Delhi
Name: Source, Length: 10683, dtype: object

### Destination

In [49]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

In [51]:
(
    flights
    .Destination
    .replace("Banglore", "Bangalore")
    .unique()
)

array(['New Delhi', 'Bangalore', 'Cochin', 'Kolkata', 'Delhi',
       'Hyderabad'], dtype=object)

### Route
This feature is redundant since this information is there in source, destination and total stops

### Departure time

In [66]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

- Checking if all the data points have only numbers and colon and nothing else

In [68]:
(
    flights
    .Dep_Time
    .loc[lambda ser:ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

- There is no invalid departure time

In [70]:
pd.to_datetime(flights.Dep_Time).dt.time

C:\Users\VAIBHAV MEHROTRA\AppData\Local\Temp\ipykernel_13188\1056054517.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival Time

In [100]:
(
    flights
    .Arrival_Time
    .loc[lambda ser:ser.str.contains("[^0-9:]")]
    .str.split(" ", n=1)
    .str.get(1)
)

0        22 Mar
2        10 Jun
6        13 Mar
7        02 Mar
8        13 Mar
          ...  
10666    13 Jun
10667    13 Mar
10672    28 Jun
10673    28 May
10674    13 Mar
Name: Arrival_Time, Length: 4335, dtype: object

### Duration

In [113]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("m")]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [115]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]
)

6474    5m
Name: Duration, dtype: object

In [116]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


- Clearly an invalid entry, hence will be removed

In [139]:
(
    flights
    .drop(index = [6474])
    .Duration
    .str.split(" ", expand = True)
    .set_axis(["hours", "minutes"], axis=1)
    .assign(
        hours = lambda df_: (
            df_
            .hours
            .str.replace("h", "")
            .astype(int)
            .mul(60)
        ),
        minutes = lambda df_:(
            df_
            .minutes
            .str.replace("m", "")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
    # .rename("Duration_minutes")
    # .to_frame()
    # .join(flights.Duration.drop(index=6474))
    # .dtypes
    # .isna().sum()
)

,Duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


### Total Stops

In [150]:
(
    flights
    .Total_Stops
    .unique()
)

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [157]:
(
    flights
    .Total_Stops
    .str.replace("non-stop", "0")
    .str.replace(" stops?", "", regex=True)
    .pipe(lambda ser: pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

### Additional Info

In [163]:
(
    flights
    .Additional_Info
    .str.replace("No info", "No Info")
    .unique()
)

array(['No Info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

## 5. Cleaning Operations

In [140]:
def convert_to_minutes(ser):
    return (
        ser
        .str.split(" ", expand = True)
        .set_axis(["hours", "minutes"], axis=1)
        .assign(
            hours = lambda df_: (
                df_
                .hours
                .str.replace("h", "")
                .astype(int)
                .mul(60)
            ),
            minutes = lambda df_:(
                df_
                .minutes
                .str.replace("m", "")
                .fillna("0")
                .astype(int)
            )
        )
        .sum(axis=1)
    )

In [168]:
def clean_data(df):
    return (
        df
        .drop(index = [6474])
        .drop_duplicates()
        .assign(**{
            col: df[col].str.strip()
            for col in flights.select_dtypes(include = "O").columns
        })
        .rename(columns = str.lower)
        .assign(
            airline = lambda df_ : (
                df_
                .airline
                .str.replace(" Premium economy", "")
                .str.replace(" Business", "")
                .str.replace("GoAir", "Go Air")
                .str.title()
            ),
            date_of_journey	= lambda df_: (
                pd.to_datetime(df_.date_of_journey, format='%d/%m/%Y')                
            ),
            source = lambda df_: (
                df_
                .source
                .replace("Banglore", "Bangalore")
            ),
            destination = lambda df_:(
                df_
                .destination
                .replace("Banglore", "Bangalore")
            ),
            dep_time = lambda df_:pd.to_datetime(df_.dep_time).dt.time,
            arrival_time = lambda df_:pd.to_datetime(df_.arrival_time).dt.time,
            duration = lambda df_ : df_.duration.pipe(convert_to_minutes),#pipe function me argument is a function that accepts a series/dataframe depending on what it is applied on
            #more efficient than apply method
            total_stops = lambda df_: (
                df_
                .total_stops
                .str.replace("non-stop", "0")
                .str.replace(" stops?", "", regex=True)
                .pipe(lambda ser: pd.to_numeric(ser))
            ),
            additional_info = lambda df_:(
                df_
                .additional_info
                .str.replace("No info", "No Info")
            )
        )
        .drop(columns = "route")
        
    )

In [169]:
flights_cleaned = clean_data(flights)#.destination.unique()#.source.unique()#.dtypes
flights_cleaned

C:\Users\VAIBHAV MEHROTRA\AppData\Local\Temp\ipykernel_13188\1572147202.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time = lambda df_:pd.to_datetime(df_.dep_time).dt.time,
C:\Users\VAIBHAV MEHROTRA\AppData\Local\Temp\ipykernel_13188\1572147202.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time = lambda df_:pd.to_datetime(df_.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Bangalore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Bangalore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Bangalore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Bangalore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Bangalore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Bangalore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Bangalore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Bangalore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


## 6. Split the Data

In [173]:
# To save computing time on sagemaker
flights_final = flights_cleaned.sample(1000)

In [174]:
X = flights_final.drop(columns = "price")
y = flights_final.price.copy()

In [175]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size = 0.2, random_state = 42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(640, 9) (640,)
(160, 9) (160,)
(200, 9) (200,)


## 7. Export the data

In [176]:
def export_data(X, y, name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR,DATA_DIR, file_name)

    X.join(y).to_csv(file_path, index = False)
    return pd.read_csv(file_path).head()

In [177]:
export_data(X_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-24,Bangalore,Delhi,07:10:00,10:05:00,175,0.0,No Info,4823
1,Multiple Carriers,2019-06-06,Delhi,Cochin,08:00:00,21:00:00,780,1.0,No Info,13587
2,Jet Airways,2019-05-09,Kolkata,Bangalore,17:00:00,18:00:00,1500,1.0,No Info,10624
3,Jet Airways,2019-06-24,Delhi,Cochin,02:15:00,19:00:00,1005,1.0,In-flight meal not included,10262
4,Multiple Carriers,2019-05-15,Delhi,Cochin,03:50:00,19:00:00,910,1.0,No Info,10197


In [178]:
export_data(X_val, y_val, "val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-04-18,Bangalore,Delhi,20:00:00,22:50:00,170,0.0,No Info,4823
1,Air India,2019-05-15,Kolkata,Bangalore,05:50:00,12:30:00,1840,2.0,No Info,12723
2,Indigo,2019-03-03,Kolkata,Bangalore,09:35:00,12:20:00,165,0.0,No Info,6565
3,Air India,2019-03-03,Delhi,Cochin,20:00:00,19:15:00,1395,2.0,No Info,17266
4,Jet Airways,2019-05-06,Kolkata,Bangalore,16:30:00,04:40:00,730,1.0,In-flight meal not included,8586


In [179]:
export_data(X_test, y_test, "test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-24,Delhi,Cochin,18:15:00,12:35:00,1100,1.0,In-flight meal not included,10262
1,Multiple Carriers,2019-05-15,Delhi,Cochin,08:30:00,19:00:00,630,1.0,No Info,9627
2,Jet Airways,2019-06-06,Delhi,Cochin,09:40:00,19:00:00,560,2.0,In-flight meal not included,10368
3,Spicejet,2019-06-09,Mumbai,Hyderabad,22:45:00,00:15:00,90,0.0,No check-in baggage included,1965
4,Indigo,2019-06-15,Delhi,Cochin,18:35:00,01:30:00,415,1.0,No Info,5883
